# **API & WEBHOOK-BASED DATA RETRIEVAL**

### **1) REQUIREMENTS SETUP**

In [11]:
# !pip install -r requirements.txt

In [12]:
import pandas as pd
import numpy as np

### **2) MODULES IMPORT**

In [29]:
from FRED_module import fetch_FRED
from EUROSTAT_module import fetch_EUROSTAT
from WB_module import fetch_WB 
from YFINANCE_module import fetch_YFINANCE
from DBNOMICS_module import fetch_DBNOMICS

### **3) DATA FETCHING**

#### **3.1) EUROSTAT-extracted indicators**

In [ ]:
# Export to the US by EU member state (monthly, total, millions of EUR)
# EU27 (from 2020) trade by SITC product group (Trade value in million ECU/EURO, Standard International Trade Classification (SITC Rev. 4, 2006): Total - all products)
# https://ec.europa.eu/eurostat/databrowser/view/ds-059331/legacyMultiFreq/table?lang=en 

# EURO_expus_m_raw = 

In [ ]:
# Industrial Production by EU member state (monthly, total manufacturing, Index: 2021=100, non-seasonally adjusted, 1996-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/sts_inpr_m/default/table?lang=en
EURO_indprod_m_raw = fetch_EUROSTAT(
    "sts_inpr_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "I21",
    "nace_r2": "C"
    }
)

EURO_indprod_m_raw = EURO_indprod_m_raw[[ "geo", "time", "value" ]]
EURO_indprod_m_raw = EURO_indprod_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Indprod Index (I21)"
})

EURO_indprod_m_raw.tail()

,Country,Time,Indprod Index (I21)
8971,SK,2025-04,102.5
8972,SK,2025-05,101.8
8973,SK,2025-06,100.2
8974,SK,2025-07,87.7
8975,SK,2025-08,86.4


In [58]:
# Unemployment rate by EU member state (monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1994-01 (varying), 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
EURO_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "PC_ACT"
    }
)

EURO_unem_m_raw = EURO_unem_m_raw[[ "geo", "time", "value" ]]
EURO_unem_m_raw = EURO_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

EURO_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
98345,SK,2025-04,13.9
98346,SK,2025-05,14.3
98347,SK,2025-06,15.4
98348,SK,2025-07,15.9
98349,SK,2025-08,15.8


In [56]:
# US Unemployment rate (monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1983-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
US_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": "US",
    "s_adj": "NSA",
    "unit": "PC_ACT"
    }
)

US_unem_m_raw = US_unem_m_raw[[ "geo", "time", "value" ]]
US_unem_m_raw = US_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

US_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
4603,US,2025-04,8.5
4604,US,2025-05,9.7
4605,US,2025-06,11.4
4606,US,2025-07,10.8
4607,US,2025-08,10.7


#### **3.5) OTHERS-extracted indicators**

In [ ]:
# Tariffs ratio